In [ ]:
#imports

import pandas as pd
import re
from textblob import TextBlob
import numpy as np
from scipy.stats import shapiro


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from scipy.stats import ttest_rel
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

from imblearn.under_sampling import RandomUnderSampler

# Retrieving and preprocessing the data

## Spotify playlist and track retrieval

## Genius lyrics retrieval

## Spotify song audio features retrieval

# Data exploration

## General exploration

In [ ]:
# giannis

## Apriori analysis

In [ ]:
# giannis

# Lyric sentiment analysis

## Data labeling

In [ ]:
#artemis

## Classification Modeling

# Audio feature based clustering

In [ ]:
#georgia & konstantina